In [ ]:
!pip install torchvision
!pip install wandb

### Dependencies

In [25]:
import torch
from torch import nn
import cv2
import numpy as np
import matplotlib.pyplot as plt
from torch.cuda import amp
import torch.optim as optim
import torchvision
import torch.nn.functional as F


# from models import Generator, Discriminator
import gc

from tqdm import tqdm
import os
import random
import sys
import time
import copy
from collections import defaultdict

# For colored terminal text
from colorama import Fore, Back, Style
c_  = Fore.GREEN
sr_ = Style.RESET_ALL

### Login to Weights & Biases

In [5]:
import wandb

API_KEY = "ad0d01e8032f3213c20796c04308d00d56de00dc"

wandb.login(key=API_KEY)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.2.1) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/hghallab/.netrc


True

In [6]:
class CFG:
    seed          = 42
    debug         = False
    model_name    = 'Resnet18Unet-pretrainedImageNet|BCELoss|250000samples'
    loss_function = 'BCELoss'
    train_bs      = 16
    valid_bs      = train_bs*2
    img_size      = [720, 1280]
    input_size    = [736, 1280]
    epochs        = 15
    lr            = 2e-3
    n_accumulate  = max(1, 32//train_bs)
    num_classes   = 1
    device        = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [7]:
print(CFG.device)

cpu


In [9]:
def set_seed(seed = 42):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)
    print('> SEEDING DONE')
    
set_seed(CFG.seed)

> SEEDING DONE


### Data Loading


### Model 

Definition of Generator and Discriminator. 

As described in the paper, the 2 networks follows the classical implementation of a Cycle Gan, in addition to a classification block inspired from MTGAN.

In [29]:
# Residual Block used in the Generator

class ResidualBlock(nn.Module):
    def __init__(self, in_features):
        super(ResidualBlock, self).__init__()

        conv_block = [  nn.ReflectionPad2d(1),
                        nn.Conv2d(in_features, in_features, 3),
                        nn.InstanceNorm2d(in_features),
                        nn.ReLU(inplace=True),
                        nn.ReflectionPad2d(1),
                        nn.Conv2d(in_features, in_features, 3),
                        nn.InstanceNorm2d(in_features)  ]

        self.conv_block = nn.Sequential(*conv_block)

    def forward(self, x):
        return x + self.conv_block(x)


# Generator

class Generator(nn.Module):
    def __init__(self, input_nc, output_nc, n_residual_blocks=9):
        super(Generator, self).__init__()

        # Initial convolution block       
        model = [   nn.ReflectionPad2d(3),
                    nn.Conv2d(input_nc, 64, 7),
                    nn.InstanceNorm2d(64),
                    nn.ReLU(inplace=True) ]
        
        # Encoder
        in_features = 16
        out_features = in_features*2
        for _ in range(2):
            model += [  nn.Conv2d(
                            in_channels=in_features,
                            out_channels=out_features,
                            kernel_size=3,
                            stride=2,
                            padding=1),
                        nn.InstanceNorm2d(out_features),
                        nn.ReLU(inplace=True) ]
            in_features = out_features
            out_features = in_features*2

        # Residual blocks
        for _ in range(n_residual_blocks):
            model += [ResidualBlock(in_features)]

        # Decoder
        out_features = in_features//2
        for _ in range(2):
            model += [  nn.ConvTranspose2d(
                                    in_features,
                                    out_features,
                                    kernel_size=3,
                                    stride=2,
                                    padding=1,
                                    output_padding=1),
                        nn.InstanceNorm2d(out_features),
                        nn.ReLU(inplace=True) ]
            in_features = out_features
            out_features = in_features//2

        # Output layer
        model += [  nn.ReflectionPad2d(3),
                    nn.Conv2d(64, output_nc, 7),
                    nn.Tanh() ]

        self.model = nn.Sequential(*model)

    def forward(self, x):
        return self.model(x)
    


# Discriminator

class Discriminator(nn.Module):
    def __init__(self, input_nc):
        super(Discriminator, self).__init__()
    
        # Convolutional layers
        self.conv1 = nn.Sequential(
            nn.Conv2d(input_nc, 64, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True))

        self.conv2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.InstanceNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True))
        
        self.conv3 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1),
            nn.InstanceNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True))
        
        self.conv4 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1),
            nn.InstanceNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True))
        
        self.conv5 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=4, stride=1, padding=1),
            nn.InstanceNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True))
        
        # Classification layers
        # Fake:Real branch
        self.conv61 = nn.Conv2d(512, 1, kernel_size=4, stride=1, padding=1)
        # Classification branch
        self.conv62 = nn.Conv2d(512, 1, kernel_size=1, stride=1, padding=1)
        
        self.avgpool = nn.AdaptiveAvgPool2d(1)
        
        self.fc = nn.Sequential(
            nn.Linear(512, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 1),
            nn.LeakyReLU(0.2, inplace=True))
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)        
        x = self.conv4(x)
        x = self.conv5(x)

        x1 = self.conv61(x1)
        x1 = F.avg_pool2d(x1, x1.size()[2:]).view(x1.size()[0], -1)

        x2 = self.conv62(x)
        x2 = self.avgpool(x2)
        x2 = x2.view(x2.size(0), -1)
        x2 = self.fc(x2)

        return x1, x2


Since we're working with a cycle GAN, we need 2 generators and 2 discriminators

In [27]:
# Paramerters definition
input_nc = 16
output_nc = 16

In [30]:
# Networks
net_G = Generator(input_nc, output_nc)
net_F = Generator(output_nc, input_nc)
net_Dx = Discriminator(input_nc)
net_Dy = Discriminator(output_nc)


In [19]:
if torch.cuda.is_available():
    net_G.cuda()
    net_F.cuda()
    net_Dx.cuda()
    net_Dy.cuda()

### Loss function

In the paper, the loss is defined as a linear combination of 4 different sub losses:
$$Loss = L_{GAN} + \lambda L_{cyc} + \alpha L_{ide} + \mu L_{cls}$$

$L_{GAN}$ being the Adversial Loss, $L_{cyc}$ the Cycle Consistency Loss, $L_{ide}$ the Identity Loss, and finally $L_{cls}$, the Classification Loss.

In [31]:
# Define weights
lamda, alpha, mu = 10, 0.5, 0.5

In [32]:
# Adversial loss
GAN_loss = nn.BCELoss()

# Cycle consitency loss
CYC_loss = nn.L1Loss()

# Identity loss
IDE_loss = nn.L1Loss()

# Classification loss
CLS_loss = nn.CrossEntropyLoss()